In this notebook I will analyze the patents about electrically debondable adhesives form Henkel.

In [1]:
# Imports 

# Own Packages
from Masterarbeit_utils.model_utils_agg import get_tokenizer, load_and_modify_model, load_pretrained_Tokenizer

# Site Packages
import pandas as pd
import numpy as np
import pickle as pk
import torch
import os 
import sys
import psutil
from collections import Counter
import itertools
# Dimension reduction algorithms
#from cuml.manifold import TSNE
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from scipy.spatial import distance
from scipy.fft import fft, fftfreq
# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import export_png
from bokeh.palettes import Viridis256, Category20
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.colors import RGB

# Huggingface
from transformers import AutoTokenizer, OPTForCausalLM
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset

output_notebook()

2023-10-02 09:21:12.014886: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 09:21:12.036342: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 09:21:13.274522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading BokehJS ...

In [2]:
###########################################################
# Loading the Henkel Patents
###########################################################

# Directories in which data important for the notebook is stored
dump_dir = 'PK_DUMP'
data_dir = 'data'

# Loading the dataframes with the patents deemed most important for electrically debondable adhesives from Henkel
henkel_patents = pd.read_csv(f'{data_dir}/Henkel_patente_patstat_docdb_families_abstract.csv', delimiter=',').reset_index(drop=True)
henkel_orbit = pd.read_csv(f'{data_dir}/Henkel_Orbit_Suche_Patstat_Export.csv', delimiter=',')

# Filtering the Samples which contain F-Terms
henkel_filtered = henkel_patents[henkel_patents['fterms'].notna()]
henkel_filtered = henkel_filtered.reset_index(drop=True)

orbit_filtered = henkel_orbit[henkel_orbit['fterms'].notna()]
orbit_filtered = orbit_filtered.reset_index(drop=True)

print(f"There are {len(henkel_patents['doc_db_family_id'].unique())} unique patents in the Henkel dataset, only {len(henkel_filtered['doc_db_family_id'].unique())} of them contain F-Terms.")

# extracting all f-terms form the datasets
fterms_henkel = [fterm[:10] for fterms in henkel_filtered['fterms'] for fterm in fterms.split(',')]
fterms_orbit = [fterm[:10] for fterms in orbit_filtered['fterms'] for fterm in fterms.split(',')]

# Aggreagting the F-Terms
with open(f'{dump_dir}/aggregation_dict_new.pk', 'rb') as f:
    aggregation_dict = pk.load(f)

def aggregate(f_term):
    try:
        return aggregation_dict[f_term]
    except KeyError:
        pass

fterms_henkel_agg = [aggregate(fterm) for fterm in fterms_henkel if aggregate(fterm) is not None]
fterms_orbit_agg = [aggregate(fterm) for fterm in fterms_orbit if aggregate(fterm) is not None]

# Counting the occurrences of the henkel and orbit fterms
counter_henkel = Counter(fterms_henkel_agg)
counter_orbit = Counter(fterms_orbit_agg)

# Structuring the henkel F-Terms
henkel_dict = {}
for fterm in counter_henkel.keys():
    theme = fterm[:5]
    try:
        _ = henkel_dict[theme]
    except KeyError:
        henkel_dict[theme] = {}

    vp = fterm[:8]
    try: 
        henkel_dict[theme][vp].append(fterm)
    except KeyError:
        henkel_dict[theme][vp] = [fterm]


################################################################
# Loading the Model
################################################################

model_name = 'gal_125_new_1'
checkpoint = int(2*86515)
# If True normalization is applied to the embeddings
norm = True
context_less = False

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/{model_name}'
os.makedirs(model_folder, exist_ok=True)


# Folder in which the tokenizer will be saved
tokenizer_folder = f'data/tokenizers/{model_name}'
os.makedirs(tokenizer_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
if model_name.split('_')[1] == '125':
    base_model_name = 'mini'
elif model_name.split('_')[1] == '1300':
    base_model_name = 'base'

# All new Torch-objects will be by default in this dtype
# if default_type = float16 fp16 must be False
default_dtype = torch.float32
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cpu'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False


###########################
# Loading the Model
###########################
device_map=None
max_memory = {}
if num_gpus > 0:
    # based on https://github.com/huggingface/accelerate/blob/5315290b55ea9babd95a281a27c51d87b89d7c85/src/accelerate/utils/modeling.py#L274
    for i in range(num_gpus):
        _ = torch.tensor([0], device=i)
    for i in range(num_gpus):
        max_memory[i] = torch.cuda.mem_get_info(i)[0]
    device_map = "auto"
max_memory["cpu"] = psutil.virtual_memory().available

model = OPTForCausalLM.from_pretrained(f'{model_folder}/checkpoint-{checkpoint}', torch_dtype=default_dtype, low_cpu_mem_usage=True,
                                               device_map=device_map, max_memory=max_memory)

###########################
# Loading the Tokenizer
###########################
tokenizer = AutoTokenizer.from_pretrained(tokenizer_folder)
n_f_terms = len(tokenizer) - tokenizer.vocab_size
print('Loaded Tokenizer from serialized instance!')    
print(f'There are {n_f_terms:,} different F-Terms in the whole Dataset!')


###########################
# Loading Descriptions
###########################
with open(f'{dump_dir}/agg_themes_descriptions_new.pk', 'rb') as f:
    theme_dict = pk.load(f)
with open(f'{dump_dir}/agg_viewpoints_descriptions_new.pk', 'rb') as f:
    viewpoint_dict = pk.load(f)
with open(f'{dump_dir}/agg_numbers_descriptions_new.pk', 'rb') as f:
    number_dict = pk.load(f)
with open(f'{dump_dir}/agg_full_descriptions_new.pk', 'rb') as f:
    full_descriptions_dict = pk.load(f)


###########################
# Extracting the Embeddings
###########################

# Extracting the classification Head weights
inp_emb = model.get_input_embeddings()


#Embeddings if the model is not a sequence classification model
out_emb = model.get_output_embeddings()
out_emb = next(out_emb.parameters()).to('cpu').detach().numpy()[2:]
inp_emb = inp_emb(torch.arange(len(tokenizer))).to('cpu').detach().numpy()[50002:]

if context_less:
    # Extracting context less embeddings
    if not os.path.isfile(f'{model_folder}/context_less_emb{checkpoint}.pk'):
        print('Calculating context less embeddings!')
        context_less_emb = [[] for _ in range(len([1 for _ in model.parameters()]))]
        for i in range(len(tokenizer)):
            print(i, end='\r')
            out = model(input_ids= torch.tensor([[i]]), attention_mask = torch.tensor([[1]]), output_hidden_states=True)
                
            out = out.hidden_states
            for i, k in enumerate(out):
                context_less_emb[i].append(k.to('cpu').detach().numpy())
        with open(f'{model_folder}/context_less_emb{checkpoint}.pk', 'wb') as f:
            pk.dump(context_less_emb, f)
    else:
        print('Loading context less embeddings from disk')
        with open(f'{model_folder}/context_less_emb{checkpoint}.pk', 'rb') as f:
            context_less_emb = pk.load(f)
        
    # Combining context less embeddings of a layer to a single tensor
    for i, layer in enumerate(context_less_emb):
        layer = [e[0] for e in layer]
        layer = np.concatenate(layer, 0)
        context_less_emb[i] = layer

# Normalizing the embeddings 
def normalize(tensor):
    if norm:
        return torch.nn.functional.normalize(torch.tensor(tensor), p=2).numpy()
    else:
        return tensor

out_emb = normalize(out_emb)
inp_emb = normalize(inp_emb)

if context_less:
    context_less_emb = [normalize(layer) for layer in context_less_emb]

# Extracting the matching F_terms for the weights and creating lists with the defintions
tokens = [tokenizer.decode(i) for i in range(len(tokenizer))]
f_term_tokens = tokens[50002:]

# Creating  a dict with f-Terms and their embedding vectors:
out_emb_dict = {token[:-1]: vec for token, vec in zip(f_term_tokens, out_emb)}
ft_emb_dict = {key: np.abs(fft(value)) for key, value in out_emb_dict.items()}
inp_emb_dict = {token[:-1]: vec for token, vec in zip(f_term_tokens, inp_emb)}
    
# Creating Context Less Embedding Dicts
if context_less:
    context_less_dicts = []
    for layer in context_less_emb:
        context_less_dicts.append({token[:-1]: vec for token, vec in zip(tokens, layer)})

# Extracting the emb_dim
for e in out_emb_dict.values():
    break
emb_dim = e.shape[-1]
print('Embedding Dimension: ', emb_dim)

There are 34 unique patents in the Henkel dataset, only 15 of them contain F-Terms.
Loaded Tokenizer from serialized instance!
There are 195,617 different F-Terms in the whole Dataset!
Embedding Dimension:  768


# Search in Theme Combinations

In [3]:
def create_fterm_dict():
    """
    Creates a hirachical dict with all F-Terms ordered by Theme -> Viewpoint > F-Terms
    """
    f_term_dict = {}
    for f_term in f_term_tokens:
        theme = f_term.split('/')[0]
        vp = f_term[:8]
        # Creating a dict entry for the theme
        try: 
            _ = f_term_dict[theme]
        except KeyError:
            f_term_dict[theme] = {}
    
        # Creating a dict entry for the viewpoint
    
        try:
            # The first dict call will def. work the second may work if the vp-dict entry 
            # was already made. If it works the theme is appended to the viewpoint dict
            f_term_dict[theme][vp].append(f_term)
        except KeyError:
            f_term_dict[theme][vp] = []

    return f_term_dict


def extract_theme_fterms(t_dict):
    fterms = []
    for vp_list in t_dict.values():
        fterms.extend(vp_list)
    return fterms        


def create_all_diffs(emb=out_emb_dict):
    """
    Creates all possible in viewpoint combinations and returns them a s a 
    """
    all_diffs = {}
    # Calculating the needed combinations
    f_term_dict = create_fterm_dict()
    for i, (theme, t_dict) in enumerate(f_term_dict.items()):
        print(i, theme, end='\r')
        all_diffs[theme] = {}
        fterms = extract_theme_fterms(t_dict)
        combinations = itertools.combinations(fterms, 2)
        for fterm1, fterm2 in combinations:
            viewpoint = fterm1[:8]
            diff = emb[fterm2[:10]] - emb[fterm1[:10]]
            diff = normalize(np.array([diff]))
            try:
                all_diffs[theme][viewpoint][(fterm1, fterm2)] = diff
            except KeyError:
                all_diffs[theme][viewpoint] = {}
                all_diffs[theme][viewpoint][(fterm1, fterm2)] = diff
    return all_diffs
    

def create_diffs_tensor(block_theme, all_diffs):
    """
    Creates a tensor with all diffs, which do not contain the block theme.
    Additionaly also returns a list with all comination descriptions
    """
    # Filtering out the unwanted theme
    diffs = {theme: t_dict for theme, t_dict in all_diffs.items() if theme != block_theme}
    out_diffs = []
    out_desc = []
    for i, (theme, t_dict) in enumerate(diffs.items()):
        #print(i, theme, end='\r')
        for vp_dict in t_dict.values():
            for comb, diff in vp_dict.items():
                out_desc.append(comb)
                out_diffs.append(diff)

    out_diffs = np.array(out_diffs)
    out_diffs = out_diffs.squeeze(1)
    return out_diffs, out_desc


def search_cos(query_vec, vecs):
    """
    Computes the cosine similarities between all_vecs and the query_vec 
    """
    cos = torch.nn.CosineSimilarity(dim=1)
    # Creating an array of query vectors, with the same number of vectors as the all_vecs array.
    query = np.concatenate([query_vec for _ in vecs], 0)

    vecs = torch.tensor(vecs, requires_grad=False)
    query = torch.tensor(query, requires_grad=False)
    simis = cos(vecs, query)
    return simis

def search_in_all(fterm1, fterm2, all_diffs, step=100):
    """
    This function computes the most similar combinations in steps of 'step' themes at a time
    """
    
    theme = fterm1[:5]
    query = out_emb_dict[fterm2[:10]] - out_emb_dict[fterm1[:10]]
    query = normalize(np.array([query]))
    iterations = -(len(all_diffs)//-step)    # Ceiling devision
    simis = []                               # Stores all computed cosine similarities
    descs = []                               # Stores all combination descriptions (fterm1, fterm2)
    
    for i in range(iterations):
        diffs_chunk = dict([d for d in all_diffs.items()][i*step: (i+1)*step])
        search_diffs, search_descs = create_diffs_tensor(theme, diffs_chunk)
        simis_chunk = search_cos(query, search_diffs)
        simis.extend(simis_chunk)
        descs.extend(search_descs)

    # Sorting for highest similarity
    idx = np.argsort(simis)[::-1]
    simis = [simis[i].item() for i in idx]
    descs = [descs[i] for i in idx]
    return simis,  descs,  idx
        
        

In [4]:
if os.path.isfile(f'{model_folder}/all_theme_diffs.pk'):
    with open(f'{model_folder}/all_theme_diffs.pk', 'rb') as f:
        all_diffs = pk.load(f)

else:
    all_diffs = create_all_diffs()
    with open(f'{model_folder}/all_theme_diffs.pk', 'wb') as f:
        pk.dump(all_diffs, f)

In [5]:
#####################################################################
# Generating in Theme combinations of the three most frequent Henkel Themes
#####################################################################


# Extracting the most important Themes
theme_occurrences = {}
for fterm, occ in counter_henkel.items():
    theme = fterm[:5]
    try:
        theme_occurrences[theme] += occ
    except KeyError:
        theme_occurrences[theme] = occ

main_themes = ['4J040', '4F100', '4J004']

# Extracting the most important F-Terms
main_fterms = list(set([fterm for fterm in fterms_henkel_agg if fterm[:5] in main_themes]))

# Creating all in Theme Combinations
combinations = []
for theme in main_themes:
    fterms = [fterm for fterm in main_fterms if fterm[:5] == theme]
    # Permutations instead of combinations because we want both directions
    combs = list(itertools.permutations(fterms, 2))
    combinations.append(combs)
    print(len(combs))



1056
1190
272


In [ ]:
# Searching with all combinations
# !!! This takes more than 24 hours, all results above the threshold similarity will be saved.
threshold = 0.2
os.makedirs(f'{model_folder}/comb_sims', exist_ok=True)
for c in combinations:
    for i, (fterm1, fterm2) in enumerate(c):
        print(i, fterm1, fterm2)
        simis, desc, idx = search_in_all(fterm1, fterm2, all_diffs, step=50)
        print('calculated')
        desc = [d for d, s in zip(desc, simis) if s > threshold]
        simis = [s for s in  simis if s > threshold]
        if len(simis) == 0: continue
        print(len(desc), len(simis))
        with open(f'{model_folder}/comb_sims/{fterm1.replace("/", "")}_{fterm2.replace("/","")}_simis.pk', 'wb') as f:
            pk.dump(simis, f)
        with open(f'{model_folder}/comb_sims/{fterm1.replace("/", "")}_{fterm2.replace("/","")}_desc.pk', 'wb') as f:
            pk.dump(desc, f)
        #with open(f'{model_folder}/comb_sims/{fterm1.replace("/", "")}_{fterm2.replace("/","")}_idx.pk', 'wb') as f:
        #    pk.dump(idx, f)

In [7]:
#########################################################################
# Searching for the combinations with the highest resulting Similarities
#########################################################################

def generate_search_results(thresh = 0.25):
    top_search_res = {}
    for c in combinations:
        theme = c[0][0][:5]
        top_search_res[theme] = {}
        for i, (fterm1, fterm2) in enumerate(c):
            print(i, end='\r')
            try: 
                with open(f'{model_folder}/comb_sims/{fterm1.replace("/", "")}_{fterm2.replace("/","")}_simis.pk', 'rb') as f:
                    simis = pk.load(f)
                with open(f'{model_folder}/comb_sims/{fterm1.replace("/", "")}_{fterm2.replace("/","")}_desc.pk', 'rb') as f:
                    desc = pk.load(f)
            except FileNotFoundError:
                continue
            
            idx = np.argsort(simis)[::-1]
            desc = [desc[i] for i in idx if simis[i] > thresh]
            simis = [simis[i] for i in idx if simis[i] > thresh]
            
            if len(desc) > 0:
                top_search_res[theme][(fterm1, fterm2)] = [desc, simis] 
        
    for theme, theme_combs in top_search_res.items():
        print(theme_dict[theme])
        print(theme, 'Number of combinations', len(theme_combs))
    return top_search_res


#####################################
# Searching for viable F-Term Chains
#####################################
def find_chains(combs, n=3):
    elements = []
    [elements.extend(comb) for comb in combs]
    elements = list(set(elements))
    chain_items = itertools.combinations(elements, n)
    chains = []
    for items in chain_items:
        chain = []
        for i in range(len(items) - 1):
            chain_element = items[i: i+2]
            if chain_element in combs:
                chain.append(chain_element)
        # Only appends chain if it is valid
        if len(chain) == n-1:
            chains.append(chain)

    # Adding cross links to the chains if there are viable cross links
    for chain in chains:
        chain_elements = [c for el in chain for c in el]
        cross_links = itertools.combinations(chain_elements, 2)
        for cross_link in cross_links:
            if cross_link in combs:
                chain.append(cross_link)
    return chains

    
def find_overlap(chain, top_search_res, n=5, chain_n=3):
    """
    n indicates the indices of the f_term string which will be used to measure the search result overlap 
    (5 = theme, 8 = viewpoint and 10 = whole fterm)
    """
    theme = chain[0][0][:5]
    l = len(chain)
    combs = {element: top_search_res[theme][element] for element in chain}
    # all fterms found in any search
    unique_values = list(set([fterm[:n] for desc, simis in combs.values() for pair in desc for fterm in pair]))
    found_by = {u_v:[] for u_v in unique_values}
    score = {u_v: [] for u_v in unique_values}
    for (q1, q2), (desc, simis) in combs.items():
        #print('Simis', simis, desc)
        for (ft1, ft2), simi in zip(desc, simis):
            found_by[ft1[:n]].extend([q1, q2])
            found_by[ft2[:n]].extend([q1, q2])
            score[ft1[:n]].append(simi)
            score[ft2[:n]].append(simi)

    # filtering for values that were found by all chain edges
    found = [u_v for u_v, q in found_by.items() if len(set(q)) == chain_n]
    score = [max(score[u_v]) for u_v in found]
    # sorting
    idx = np.argsort(score)[::-1]
    found = [found[i] for i in idx]
    score = [score[i] for i in idx]

    return found, score

In [19]:

# generating the overlaps for all chains
top_search_res = generate_search_results(0.3)
df = {}
found_fterms = []
found_by = []
for searches in top_search_res.values():
    chains = find_chains(searches.keys(), 3)
    #print('Len Chains', len(chains))
    for chain in chains:
        themes, scores = find_overlap(chain, top_search_res, 10)
        chain_elements = [x for a in chain for x in a]
        chain = 'TRIANGLE'
        for fterm in list(set(chain_elements)):
            label = number_dict[fterm]
            chain += f'{fterm}:'
            chain += label
            chain += '|'
    
        #themes = [f'({theme}):{theme_dict[theme]}| Score: {score}' for theme, score in zip(themes, scores)]
        themes = [f'({theme}):{full_descriptions_dict[theme]}| Score: {score}' for theme, score in zip(themes, scores)]
        if len(themes) == 0:
            continue
        found_fterms.extend(themes)
        found_by.extend([chain for _ in themes])
        df[chain] = pd.Series(themes)

df = pd.DataFrame(df)          
df = df.reset_index()

#df.to_excel('Overlap_4__0_3_Fterms.xlsx', index=False)

adhesives or adhesive processes
4J040 Number of combinations 89
laminated bodies (2)
4F100 Number of combinations 138
adhesive tapes
4J004 Number of combinations 0


In [11]:
# Ploting the found F-Terms

,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/MB08:. structures or shapes of parts to which adhesives are applied|4J040/DF02:. ester homopolymers or copolymers|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/MB08:. structures or shapes of parts to which adhesives are applied|4J040/DF01:. acids salts thereof or ammonium salts including anhydrides|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/MB08:. structures or shapes of parts to which adhesives are applied|4J040/EH02:. polymers having nitrogen-containing heterocyclic rings in the main chain|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/MB08:. structures or shapes of parts to which adhesives are applied|4J040/EE01:. aliphatic polyethers|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/MB08:. structures or shapes of parts to which adhesives are applied|4J040/EK03:. polysiloxanes|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/JB09:. pressure sensitive or adhesive types|4J040/DF01:. acids salts thereof or ammonium salts including anhydrides|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/EH02:. polymers having nitrogen-containing heterocyclic rings in the main chain|4J040/JB09:. pressure sensitive or adhesive types|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/EE01:. aliphatic polyethers|4J040/JB09:. pressure sensitive or adhesive types|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/DF02:. ester homopolymers or copolymers|4J040/HC01:. amines or quaternary ammonium compounds|,TRIANGLE4J040/EB09:. condensation polymers of aldehydes or ketones with only compounds containing hydrogen bound to nitrogen|4J040/DF01:. acids salts thereof or ammonium salts including anhydrides|4J040/HC01:. amines or quaternary ammonium compounds|,...,TRIANGLE4F100/JJ03:. heat resistance being property or function|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/JK01:. strength or rigidity|,TRIANGLE4F100/JJ03:. heat resistance being property or function|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/AK01:. organic macromolecular materials|,TRIANGLE4F100/JJ03:. heat resistance being property or function|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/BA02:. with two-layer structure|,TRIANGLE4F100/JJ03:. heat resistance being property or function|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/JL11:. adhesiveness|,TRIANGLE4F100/JJ03:. heat resistance being property or function|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/EH11:. layering accompanied with new layer formation|,TRIANGLE4F100/BA04:. with four-layer structure|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/CB05:. pressure-sensitive adhesion materials|,TRIANGLE4F100/JG01:. conductivity being properties or functions|4F100/BA04:. with four-layer structure|4F100/EJ61:. electrical treatment or magnetic treatment|,TRIANGLE4F100/EJ61:. electrical treatment or magnetic treatment|4F100/BA02:. with two-layer structure|4F100/CB05:. pressure-sensitive adhesion materials|,TRIANGLE4F100/JG01:. conductivity being properties or functions|4F100/EJ61:. electrical treatment or magnetic treatment|4F100/BA02:. with two-layer structure|,TRIANGLE4F100/EJ61:. electrical treatment or magnetic treatment|4F100/CA21:. conductive materials|

In [34]:
fterms = [key for key in counter_henkel.keys()]
occ = [occ for occ in counter_henkel.values()]
idx = np.argsort(occ)[::-1]
fterms = [fterms[i] for i in idx]
occ = [occ[i] for i in idx]


for fterm, f in zip(fterms, occ):
    
    theme = fterm[:5]
    viewpoint = fterm[:8]
    if theme not in main_themes[2]:
        continue
    print(f, fterm, theme_dict[theme], viewpoint_dict[viewpoint], number_dict[fterm])
    print('')

14 4J004/AA03 adhesive tapes composition of adhesives or an adhesive article . high-molecular compounds

12 4J004/AB01 adhesive tapes adhesiveness of adhesive articles . pressure-sensitive adhesion type

9 4J004/CB03 adhesive tapes types of carriers . special films

7 4J004/CC02 adhesive tapes forms or structures of carriers(jhy) . foil-like, film like or sheet-like

7 4J004/CA07 adhesive tapes properties of carriers . inorganic materials

6 4J004/FA08 adhesive tapes applications . support or attachment (e.g., suspenders)

6 4J004/DB02 adhesive tapes forms or structures of parting layer . having a parting sheet or lining

5 4J004/EA05 adhesive tapes types of products . double sided tapes

5 4J004/AA17 adhesive tapes composition of adhesives or an adhesive article . organic low-molecular compounds

5 4J004/FA05 adhesive tapes applications . electrical use (e.g., insulating or conductive tape)

4 4J004/CA01 adhesive tapes properties of carriers . high-molecular compounds

4 4J004/EA06 ad

In [ ]:
# Show top n search results for evaluation
n = 5
for theme, combs in top_search_res.items():
    for (q1, q2), (desc, simis) in combs.items():
        print('')
        print('QUERY:')
        print( full_descriptions_dict[q1], '|||', full_descriptions_dict[q2])
        print('______________________')
        
        for (ft1, ft2), s in zip(desc[:n], simis[:n]):
            print(s, full_descriptions_dict[ft1[:10]], '|||', full_descriptions_dict[ft2[:10]])
            print('')

In [ ]:
###############################################################
# Searching for promising Henkel Combinations  (In Theme)
###############################################################


search_combination = ['4F100/JL11', '4F100/JG01']  # (adhesiveness, conductivity being properties or funcitons)
#search_combination = ['4J040/JB09', '4J040/PA21']  # (pressure sensitive adhesive or adhesive types, use of adhesive characterised by specific shapess of functions)
#search_combination = ['4J004/CC02', '4J004/CA07']  # (foil like, inorganic materials)

simis, descs, idx = search_in_all(*search_combination, all_diffs, step=500)
results = descs[:100]


In [ ]:
print('(adhesiveness, conductivity being properties or funcitons)')
for fterm1, fterm2 in results:
    theme = theme_dict[fterm1[:5]]
    vp1 = viewpoint_dict[fterm1[:8]]
    vp2 = viewpoint_dict[fterm2[:8]]
    n1 = number_dict[fterm1[:10]]
    n2 = number_dict[fterm2[:10]]

    print(f'''    
Theme: {theme}
vp1: {vp1}     vp2: {vp2}
n1: {n1}       n2:{n2}
''')

In [ ]:
# Extracting uniqe F-terms from the results
found_f_terms = []
[found_f_terms.extend(comb) for comb in results]
found_f_terms = list(set(found_f_terms))

# Embedding the found F-Terms
hits_emb = torch.tensor(np.array([out_emb_dict[fterm[:10]] for fterm in found_f_terms]))

orbit_emb = []
o_ft = []
for fterm in counter_orbit.keys(): 
    try:
        orbit_emb.append(out_emb_dict[fterm])
        o_ft.append(fterm)
    except KeyError:
        pass

orbit_emb = np.array(orbit_emb)

henkel_emb = []
h_ft = []
for fterm in search_combination:
    try:
        henkel_emb.append(out_emb_dict[fterm])
        h_ft.append(fterm)
    except KeyError:
        pass

henkel_emb = np.array(henkel_emb)

all_emb = np.concatenate([orbit_emb, henkel_emb, hits_emb], 0)

tsne = TSNE(n_components=2, verbose=0, random_state=69)
rep = tsne.fit_transform(all_emb)

datasource_henkel = ColumnDataSource(
        data=dict(
            x = rep[len(o_ft):len(o_ft) + len(h_ft),0],
            y = rep[len(o_ft):len(o_ft) + len(h_ft),1],
            fterms = h_ft,
            themes = [theme_dict[fterm[:5]] for fterm in h_ft],
            viewpoints = [viewpoint_dict[fterm[:8]] for fterm in h_ft],
            numbers = [number_dict[fterm[:10]] for fterm in h_ft]))

datasource_orbit = ColumnDataSource(
        data=dict(
            x = rep[:len(o_ft), 0],
            y = rep[:len(o_ft), 1],
            fterms = o_ft,
            themes = [theme_dict[fterm[:5]] for fterm in o_ft],
            viewpoints = [viewpoint_dict[fterm[:8]] for fterm in o_ft],
            numbers = [number_dict[fterm[:10]] for fterm in o_ft]))

datasource_emb_search = ColumnDataSource(
        data=dict(
            x = rep[len(o_ft) + len(h_ft):, 0],
            y = rep[len(o_ft) + len(h_ft):, 1],
            fterms = found_f_terms,
            themes = [theme_dict[fterm[:5]] for fterm in found_f_terms],
            viewpoints = [viewpoint_dict[fterm[:8]] for fterm in found_f_terms],
            numbers = [number_dict[fterm[:10]] for fterm in found_f_terms]))


hover_tsne = HoverTool(tooltips='<div style="font-size: 12px;"><b>F-Term:</b> @fterms<br><b>Theme:</b> @themes<br><b>Viewpoint:</b> @viewpoints<br><b>Number:</b> @numbers<br><b>Query:</b> @found_by</div>', mode='mouse')
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(width=1500, height=1500, tools=tools_tsne, title='Henkel and Orbit Embeddings')
    
plot_tsne.circle('x', 'y', size=10, fill_color=RGB(250, 50, 75), alpha=1, line_width=0, source=datasource_henkel, name="Henkel Embeddings")
plot_tsne.square('x', 'y', size=7, fill_color=RGB(50, 75, 250), alpha=0.2, line_width=0, source=datasource_orbit, name="Orbit Embeddings")
plot_tsne.triangle('x', 'y', size=7, fill_color=RGB(75, 250, 50), alpha=1, line_width=0, source=datasource_emb_search, name="Cos Similar Embeddings")

show(plot_tsne)